# POPULATION HEALTH BY URBAN EXTENTS
We have been having a look at population health based on synthetic data.  Let's now look at visualising this information using a streamlit python application.  The urban extent boundaries are available in the Snowflake Market Place - you will have access to this if you completed the previous step

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
import pydeck as pdk

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


## Curating the data

Frstly let's get the data we used before - we are going to focus on the location, the Gender, the body weight and finally each morbidity.

In [ ]:
population_health = session.table('DEFAULT_DATABASE.DEFAULT_SCHEMA.SYNTHETIC_POPULATION')

population_health = population_health.select('BODY_WEIGHT','SEX','LAT','LON','CANCER','DIABETES','COPD','ASTHMA','HYPERTENSION')


We will now bucket all the locations into **H3 indexes** which we did in the previoius excercise.  Next, a new table is created called **POPULATION_HEALTH_H3**.  

In [ ]:
population_h3 = population_health.group_by(call_function('H3_LATLNG_TO_CELL_STRING',col('LAT'),
                                         col('LON'),8).alias('H3'),
                           'BODY_WEIGHT',
                           'SEX').agg(count('*').alias('TOTAL_POPULATION')
                                      ,sum('CANCER').alias('CANCER'),
                                      sum('DIABETES').alias('DIABETES'),
                                      sum('COPD').alias('COPD'),
                                      sum('ASTHMA').alias('ASTHMA'),
                                      sum('HYPERTENSION').alias('HYPERTENSION'),
                                      avg('LAT').alias('LAT'),
                                      avg('LON').alias('LON'))

population_h3.write.mode('overwrite').save_as_table("DEFAULT_SCHEMA.POPULATION_HEALTH_H3")

population_h3 = session.table('DEFAULT_SCHEMA.POPULATION_HEALTH_H3')
population_h3

Now lets look at the built up urban areas from the datashare.  You will note that selecting the shared dataset is the same as any other table

In [ ]:
built_up_areas = session.table('URBAN_EXTENTS_FOR_CITIES_TOWNS_AND_VILLAGES__GREAT_BRITAIN_OPEN_BUILT_UP_AREAS.PRS_OPEN_BUILT_UP_AREAS_SCH.PRS_OPEN_BUILT_UP_AREAS_TBL')
built_up_areas.limit(5)

Next we need to join the two datasets together.  we need to join by the H3 index code.  Therefore, we need to split out all the towns by the same H3 index number.  You can use the function **H3_COVERAGE_STRINGS** for this

In [ ]:
coverage = built_up_areas.select('GSSCODE','NAME1_TEXT',call_function('H3_COVERAGE_STRINGS',col('GEOGRAPHY'),8).alias('H3')).cache_result()

coverage.limit(10)

You will note that there are multiple coverage strings per row.  In order to join with the population dataset, we will need to 'flatten' the table so you have one H3 index per row. The function we are using is a **table function** called **flatten**. We Flatten on the column H3.  We will then select the result which is returned as **VALUE** as well as all the original columns.  We will only select **GSSCODE** and **NAME1_TEXT**.  The Value Column is renamed as **H3**.

In [ ]:
coverage_flattened = coverage.join_table_function('flatten','H3').select('GSSCODE',
                                                                         'NAME1_TEXT',
                                                                         col('VALUE').astype(StringType()).alias('H3'))

coverage_flattened.write.mode('overwrite').save_as_table("DEFAULT_SCHEMA.EXTENTS_WITH_H3")
coverage_flattened = session.table('DEFAULT_SCHEMA.EXTENTS_WITH_H3')
coverage_flattened

Joining to the Population table should now be really easy as you have a a H3 column in the new areas dataframe and a H3 column in the population health table.  You will use the **join** function, then you choose the column to join onto.  The default type of join is an **inner join**, you can change this which will show all the areas that does not have a population.

In [ ]:
population_by_area = coverage_flattened.join(population_h3,'H3','inner')
population_by_area

## Visualising the data


### Creating Filters

Firstly, let's create a filters to the dataset.  It would be good to filter by urban name, body weight and sex.  This is where we create streamlit components.  We will create three drop downlists.  I have nested it inside a container with a pre defined height and chose to create the drop down lists in 3 columns.

Each select box is populated with distinct values from the newly created dataframe.

Next, we will filter the dataframe based on these select boxes using the **filter** function.  This actually translates to the **WHERE** clause in SQL

In [ ]:
with st.container(height=400):
    st.title('POPULATION HEALTH BY URBAN AREA')
    col1,col2,col3 = st.columns(3)
    with col1:
        urban_area = st.selectbox('Select Urban Area:', population_by_area.select('NAME1_TEXT').distinct())
    with col2:
        body_weight = st.selectbox('Select Body Weight:', population_by_area.select('BODY_WEIGHT').distinct())
    with col3:
        SEX = st.selectbox('Select Gender:', population_by_area.select('SEX').distinct())

    df = population_by_area.filter((col('SEX')==SEX)&
                                  (col('BODY_WEIGHT')==body_weight)&
                                (col('NAME1_TEXT')==urban_area))
    st.dataframe(df)
    


### Create Metrics
Now we have fitered to the right areas, let's now summarize the results and create metrics.  Again, used the columns to lay out all the metrics accross the page

In [ ]:
pop_metrics = df.agg(sum('TOTAL_POPULATION').alias('TOTAL_POPULATION'),
      sum('CANCER').alias('CANCER'),
      sum('DIABETES').alias('DIABETES'),
      sum('COPD').alias('COPD'),
      sum('ASTHMA').alias('ASTHMA'),
      sum('HYPERTENSION').alias('HYPERTENSION')).to_pandas()

col1,col2,col3,col4,col5,col6 = st.columns(6)
with col1:
    st.metric('Total Population',pop_metrics.TOTAL_POPULATION)
with col2:
    st.metric('Cancer Sufferers',pop_metrics.CANCER)
with col3:
    st.metric('Diabetics',pop_metrics.DIABETES)
with col4:
    st.metric('COPD Sufferers',pop_metrics.COPD)
with col5:
    st.metric('Asthmatics',pop_metrics.ASTHMA)
with col6:
    st.metric('Hypertension', pop_metrics.HYPERTENSION)

### Creating a map

You will be creating a H3 map to visualise the hexagons which can filter by each extent area.  NB the color is in RGB format, you will need to manipulate the RGB based on the fields in the data.  This example is using the total population field.  Further calculations might need to be considered to have further control of how the colours are presented.

In [ ]:
center = df.agg(avg('LAT'),avg('LON')).collect()

LAT = center[0][0]
LON = center[0][1]

layer = pdk.Layer(
    "H3HexagonLayer",
    df.to_pandas(),
    pickable=True,
    stroked=True,
    filled=True,
    extruded=False,
    get_hexagon="H3",
    get_fill_color="[255 - TOTAL_POPULATION, 255-TOTAL_POPULATION, 255]",
    get_line_color=[1, 1, 1],
    line_width_min_pixels=1,
)
view_state = pdk.ViewState(latitude=LAT, longitude=LON, zoom=12, bearing=0, pitch=0)
r = pdk.Deck(map_style=None,layers=[layer], initial_view_state=view_state, 
             tooltip={"html": "Total Population: {TOTAL_POPULATION}<BR>\
                                Total Cancer: {CANCER}<BR>\
                                Total Diabetes: {DIABETES}<BR>\
                                Total COPD: {COPD}<BR>\
                                Total Hypertension {HYPERTENSION}"})

st.pydeck_chart(r)

### Switching to Streamlit to view the dashboard

Now we have seen how to create basic objects for our dashbord, we will now put these objects into a **Streamlit App**.  Please navigate to **Projects > Streamlit > POPULATION_HEALTH_BY_URBAN_EXTENTS** to see this in action